<a href="https://colab.research.google.com/github/nanpolend/365E5/blob/master/%E3%80%8C%E6%9C%80%E6%96%B0%E7%BE%8E%E5%9C%8B%E5%82%B5%E5%8A%B5%E6%A8%A1%E5%9E%8B_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def hull_white_model(self, alpha=0.1, sigma=0.01):
    """Hull-White雙因子模型"""
    # 實現Hull-White隨機微分方程
    pass
def calculate_spread(self, corporate_bond_yield):
    """計算信用利差"""
    treasury_yield = self.rates_data.iloc[-1]
    return corporate_bond_yield - treasury_yield
def portfolio_var(self, bond_list, confidence=0.95):
    """債券組合風險值計算"""
    # 基於歷史模擬法計算VaR
    pass
def real_time_update(self):
    """串接Bloomberg API獲取即時數據"""
    from blp import blp
    bquery = blp.BlpQuery().start()
    data = bquery.bdh(['US10YT Index'], ['PX_LAST'], period='D')
    return data
# -*- coding: utf-8 -*-
"""
美國債券分析系統 v1.2
整合：
1. Vasicek利率模型
2. LSTM利率預測
3. 債券風險指標計算
參考模型來自：
- Bloomberg Bond Valuation Model
- Federal Reserve Economic Data (FRED) 方法論
"""

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import plotly.graph_objects as go

class BondAnalysisSystem:
    def __init__(self):
        self.rates_data = None
        self.model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def fetch_data(self, ticker='^TNX', years=10):
        """獲取歷史利率數據"""
        self.rates_data = yf.download(ticker, period=f'{years}y')['Adj Close']
        print(f"獲取{self.rates_data.name} {len(self.rates_data)}筆數據")

    def vasicek_model(self, r0=0.05, a=0.15, b=0.05, sigma=0.01, days=30):
        """
        Vasicek利率模型模擬
        dr = a(b - r)dt + σdW
        """
        dt = 1/252
        rates = [r0]
        for _ in range(days):
            dr = a*(b - rates[-1])*dt
            shock = sigma * np.sqrt(dt) * np.random.normal()
            rates.append(rates[-1] + dr + shock)
        return rates

    def bond_price(self, par, coupon, T, freq=2, r=None):
        """債券定價模型"""
        if r is None:
            r = self.rates_data.iloc[-1]/100
        periods = T*freq
        coupon_pmt = par * coupon/freq
        dt = np.arange(1, periods + 1)/freq
        cash_flows = np.full(periods, coupon_pmt)
        cash_flows[-1] += par
        return np.sum(cash_flows * np.exp(-r*dt))

    def duration_convexity(self, par, coupon, T, freq=2, delta=0.0001):
        """計算修正久期與凸性"""
        r = self.rates_data.iloc[-1]/100
        P0 = self.bond_price(par, coupon, T, freq, r)
        P_plus = self.bond_price(par, coupon, T, freq, r + delta)
        P_minus = self.bond_price(par, coupon, T, freq, r - delta)

        duration = (P_minus - P_plus)/(2 * P0 * delta)
        convexity = (P_plus + P_minus - 2*P0)/(P0 * delta**2)
        return duration, convexity

    def prepare_lstm_data(self, lookback=60):
        """準備LSTM訓練數據"""
        scaled_data = self.scaler.fit_transform(self.rates_data.values.reshape(-1,1))

        X, y = [], []
        for i in range(lookback, len(scaled_data)):
            X.append(scaled_data[i-lookback:i, 0])
            y.append(scaled_data[i, 0])
        return np.array(X), np.array(y)

    def train_lstm(self, epochs=50):
        """訓練利率預測模型"""
        X, y = self.prepare_lstm_data()
        X = X.reshape(X.shape[0], X.shape[1], 1)

        self.model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
            LSTM(50),
            Dense(1)])
        self.model.compile(optimizer='adam', loss='mse')
        self.model.fit(X, y, epochs=epochs, batch_size=32)

    def predict_rates(self, days=30):
        """預測未來利率走勢"""
        last_sequence = self.scaler.transform(
            self.rates_data[-60:].values.reshape(-1,1))

        predictions = []
        current_seq = last_sequence.copy()
        for _ in range(days):
            pred = self.model.predict(current_seq.reshape(1, 60, 1))[0,0]
            predictions.append(pred)
            current_seq = np.append(current_seq[1:], pred)

        pred_rates = self.scaler.inverse_transform(
            np.array(predictions).reshape(-1,1))
        return pred_rates.flatten()

    def stress_test(self, par, coupon, T, shocks=[-0.02, -0.01, 0.01, 0.02]):
        """壓力測試分析"""
        base_rate = self.rates_data.iloc[-1]/100
        results = []
        for shock in shocks:
            new_price = self.bond_price(par, coupon, T, r=base_rate+shock)
            results.append((shock*100, new_price))
        return pd.DataFrame(results, columns=['利率變動(bp)', '債券價格'])

    def plot_analysis(self):
        """可視化分析結果"""
        fig = go.Figure()
def hull_white_model(self, alpha=0.1, sigma=0.01):
    """Hull-White雙因子模型"""
    # 實現Hull-White隨機微分方程
    pass
def calculate_spread(self, corporate_bond_yield):
    """計算信用利差"""
    treasury_yield = self.rates_data.iloc[-1]
    return corporate_bond_yield - treasury_yield
def portfolio_var(self, bond_list, confidence=0.95):
    """債券組合風險值計算"""
    # 基於歷史模擬法計算VaR
    pass
def real_time_update(self):
    """串接Bloomberg API獲取即時數據"""
    from blp import blp
    bquery = blp.BlpQuery().start()
    data = bquery.bdh(['US10YT Index'], ['PX_LAST'], period='D')
    return data
# -*- coding: utf-8 -*-
"""
美國債券分析系統 v1.2
整合：
1. Vasicek利率模型
2. LSTM利率預測
3. 債券風險指標計算
參考模型來自：
- Bloomberg Bond Valuation Model
- Federal Reserve Economic Data (FRED) 方法論
"""

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import plotly.graph_objects as go

class BondAnalysisSystem:
    def __init__(self):
        self.rates_data = None
        self.model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def fetch_data(self, ticker='^TNX', years=10):
        """獲取歷史利率數據"""
        self.rates_data = yf.download(ticker, period=f'{years}y')['Adj Close']
        print(f"獲取{self.rates_data.name} {len(self.rates_data)}筆數據")

    def vasicek_model(self, r0=0.05, a=0.15, b=0.05, sigma=0.01, days=30):
        """
        Vasicek利率模型模擬
        dr = a(b - r)dt + σdW
        """
        dt = 1/252
        rates = [r0]
        for _ in range(days):
            dr = a*(b - rates[-1])*dt
            shock = sigma * np.sqrt(dt) * np.random.normal()
            rates.append(rates[-1] + dr + shock)
        return rates

    def bond_price(self, par, coupon, T, freq=2, r=None):
        """債券定價模型"""
        if r is None:
            r = self.rates_data.iloc[-1]/100
        periods = T*freq
        coupon_pmt = par * coupon/freq
        dt = np.arange(1, periods + 1)/freq
        cash_flows = np.full(periods, coupon_pmt)
        cash_flows[-1] += par
        return np.sum(cash_flows * np.exp(-r*dt))

    def duration_convexity(self, par, coupon, T, freq=2, delta=0.0001):
        """計算修正久期與凸性"""
        r = self.rates_data.iloc[-1]/100
        P0 = self.bond_price(par, coupon, T, freq, r)
        P_plus = self.bond_price(par, coupon, T, freq, r + delta)
        P_minus = self.bond_price(par, coupon, T, freq, r - delta)

        duration = (P_minus - P_plus)/(2 * P0 * delta)
        convexity = (P_plus + P_minus - 2*P0)/(P0 * delta**2)
        return duration, convexity

    def prepare_lstm_data(self, lookback=60):
        """準備LSTM訓練數據"""
        scaled_data = self.scaler.fit_transform(self.rates_data.values.reshape(-1,1))

        X, y = [], []
        for i in range(lookback, len(scaled_data)):
            X.append(scaled_data[i-lookback:i, 0])
            y.append(scaled_data[i, 0])
        return np.array(X), np.array(y)

    def train_lstm(self, epochs=50):
        """訓練利率預測模型"""
        X, y = self.prepare_lstm_data()
        X = X.reshape(X.shape[0], X.shape[1], 1)

        self.model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
            LSTM(50),
            Dense(1)])
        self.model.compile(optimizer='adam', loss='mse')
        self.model.fit(X, y, epochs=epochs, batch_size=32)

    def predict_rates(self, days=30):
        """預測未來利率走勢"""
        last_sequence = self.scaler.transform(
            self.rates_data[-60:].values.reshape(-1,1))

        predictions = []
        current_seq = last_sequence.copy()
        for _ in range(days):
            pred = self.model.predict(current_seq.reshape(1, 60, 1))[0,0]
            predictions.append(pred)
            current_seq = np.append(current_seq[1:], pred)

        pred_rates = self.scaler.inverse_transform(
            np.array(predictions).reshape(-1,1))
        return pred_rates.flatten()

    def stress_test(self, par, coupon, T, shocks=[-0.02, -0.01, 0.01, 0.02]):
        """壓力測試分析"""
        base_rate = self.rates_data.iloc[-1]/100
        results = []
        for shock in shocks:
            new_price = self.bond_price(par, coupon, T, r=base_rate+shock)
            results.append((shock*100, new_price))
        return pd.DataFrame(results, columns=['利率變動(bp)', '債券價格'])

    def plot_analysis(self):
        """可視化分析結果"""
        fig = go.Figure()

        # 歷史利率
        fig.add_trace(go.Scatter(
            x=self.rates_data.index,
            y=self.rates_data,
            name='歷史利率',
            line=dict(color='blue'))
        )

        # 模擬利率路徑
        simulated = self.vasicek_model()
        fig.add_trace(go.Scatter(
            x=pd.date_range(start=self.rates_data.index[-1], periods=30),
            y=simulated,
            name='Vasicek模擬',
            line=dict(color='red', dash='dot')))

        # LSTM預測
        lstm_pred = self.predict_rates()
        fig.add_trace(go.Scatter(
            x=pd.date_range(start=self.rates_data.index[-1], periods=30),
            y=lstm_pred,
            name='LSTM預測',
            line=dict(color='green')))

        fig.update_layout(
            title='利率走勢分析',
            yaxis_title='收益率(%)',
            hovermode="x unified",
            template="plotly_white")
        fig.show()

# 使用範例
if __name__ == "__main__":
    system = BondAnalysisSystem()
    system.fetch_data(ticker='^TNX', years=10)  # 10年期美債殖利率

    # 債券參數設定
    par_value = 1000       # 面值
    coupon_rate = 0.045    # 票面利率
    maturity = 5           # 剩餘年限

    # 基本分析
    price = system.bond_price(par_value, coupon_rate, maturity)
    duration, convexity = system.duration_convexity(par_value,
    coupon_name='歷史利率',line=dict(color='blue'))

def plot_analysis(self):
        """可視化分析結果"""
        fig = go.Figure()
          # 歷史利率
            fig.add_trace(go.Scatter(
            x=self.rates_data.index,
            y=self.rates_data,
            name='歷史利率',
            line=dict(color='blue')))

        # 模擬利率路徑
        simulated = self.vasicek_model() # Fixed: Corrected indentation
        fig.add_trace(go.Scatter(
        x=pd.date_range(start=self.rates_data.index[-1],periods=30),
            y=simulated,
            name='Vasicek模擬',
            line=dict(color='red', dash='dot')))
          # LSTM預測
          lstm_pred = self.predict_rates()
          fig.add_trace(go.Scatter(
            x=pd.date_range(start=self.rates_data.index[-1], periods=30),
            y=lstm_pred,
            name='LSTM預測',
          line=dict(color='green')))
          fig.update_layout(
          title='利率走勢分析',
          yaxis_title='收益率(%)',
          hovermode="x unified",
          template="plotly_white")
          fig.show()
# 使用範例
if __name__ == "__main__":
    system = BondAnalysisSystem()
    system.fetch_data(ticker='^TNX', years=10)  # 10年期美債殖利率

    # 債券參數設定
    par_value = 1000       # 面值
    coupon_rate = 0.045    # 票面利率
    maturity = 5           # 剩餘年限

    # 基本分析
    price = system.bond_price(par_value, coupon_rate, maturity)
    duration, convexity = system.duration_convexity(par_value, coupon_rate, maturity)
    print(f"\n當前債券價格：${price:.2f}")
    print(f"修正久期：{duration:.2f} 年")
    print(f"凸性：{convexity:.2f}")

    # 機器學習預測
    system.train_lstm(epochs=30)

    # 壓力測試
    stress_results = system.stress_test(par_value, coupon_rate, maturity)
    print("\n壓力測試結果：")
    print(stress_results)

    # 可視化
    system.plot_analysis()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 324)